In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
# something is bugged
# this can't be together with imports: https://github.com/jupyter/notebook/issues/3385
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300.0 # default: 72.0
mpl.rcParams['figure.figsize'] = [8., 4.5] # default: [6., 4.]

In [ ]:
df_tmp = dict()

# Project

Life expectancy

http://gapm.io/ilex

In [ ]:
df_tmp['life_expectancy'] = pd.read_csv('life_expectancy_years.csv', index_col=[0])

Population

http://gapm.io/dpop

In [ ]:
df_tmp['population'] = pd.read_csv('population_total.csv', index_col=[0])

GDP per capita (inflation adjusted)

http://gapm.io/dgdppc

In [ ]:
df_tmp['income'] = pd.read_csv('income_per_person_gdppercapita_ppp_inflation_adjusted.csv', index_col=[0])

Surface area

https://data.worldbank.org/indicator/AG.SRF.TOTL.K2

In [ ]:
df_tmp['area'] = pd.read_csv('surface_area_sq_km.csv', index_col=[0])

CO2 emissions (tonnes per person)

https://mdgs.un.org/unsd/mdg/Data.aspx

In [ ]:
df_tmp['co2'] = pd.read_csv('co2_emissions_tonnes_per_person.csv', index_col=[0])

Access to basic safe water source (percent of population)

https://cdiac.ess-dive.lbl.gov/

In [ ]:
df_tmp['h2o'] = pd.read_csv('at_least_basic_water_source_overall_access_percent.csv', index_col=[0])

Current health expenditure (percent of GDP)

https://data.worldbank.org/indicator/SH.XPD.CHEX.GD.ZS

In [ ]:
df_tmp['exp_health'] = pd.read_csv('sh_xpd_chex_gd_zs.csv', index_col=[0])

Infant mortality rate (death between birth and age 1 per 1000 live births)

https://data.worldbank.org/indicator/SP.DYN.IMRT.IN

In [ ]:
df_tmp['infant_mortality'] = pd.read_csv('sp_dyn_imrt_in.csv', index_col=[0])

Literacy rate, adult total (percent of people ages 15 and above)

https://data.worldbank.org/indicator/SE.ADT.LITR.ZS

In [ ]:
df_tmp['literacy'] = pd.read_csv('se_adt_litr_zs.csv', index_col=[0])

Polio immunization coverage among 1-year-olds (%)

https://apps.who.int/gho/data/node.imr.WHS4_544

In [ ]:
df_tmp['polio'] = pd.read_csv(
    'WHS4_544.csv',
    header=[1], skiprows=0, skipinitialspace=True, index_col=[0]
).iloc[:, ::-1]
df_tmp['polio'].rename(columns=lambda x: x.strip(), inplace=True)

BCG immunization coverage among 1-year-olds (%)

https://apps.who.int/gho/data/node.imr.WHS4_543

In [ ]:
df_tmp['bcg'] = pd.read_csv(
    'WHS4_543.csv',
    header=[1], skiprows=0, skipinitialspace=True, index_col=[0]
).iloc[:, ::-1]
df_tmp['bcg'].rename(columns=lambda x: x.strip(), inplace=True)

Hepatitis B (HepB3) immunization coverage among 1-year-olds (%)

https://apps.who.int/gho/data/node.imr.WHS4_117

In [ ]:
df_tmp['hepB'] = pd.read_csv(
    'WHS4_117.csv',
    header=[1], skiprows=0, skipinitialspace=True, index_col=[0]
).iloc[:, ::-1]
df_tmp['hepB'].rename(columns=lambda x: x.strip(), inplace=True)

In [ ]:
alcohol = pd.read_csv(
    'Recorded alcohol per capita consumption.csv',
    header=[1], skiprows=[], skipinitialspace=True, index_col=[0,1,2]
).iloc[:, ::-1]
df_tmp['alcohol'] = alcohol.xs(' All types', level=2).droplevel(level=1).rename(columns=lambda x: x.strip()).drop_duplicates()

In [ ]:
# clean up
for x in df_tmp.values():
    x.replace('', np.nan, inplace=True)
    # interpolate empty data points
    x.interpolate(method='linear', inplace=True)
    # drop all data
    x.drop(x.columns.to_series()[:'1950'], axis=1, inplace=True)
    # drop predictions
    x.drop(x.columns.to_series()['2019':], axis=1, inplace=True)
    # rename countries, because we use different data sets
    x.rename(index={
        'Czech Republic': 'Czechia',
        'United States': 'United States of America',
        'Iran (Islamic Republic of)': 'Iran',
        'Vietnam': 'Viet Nam',
        'Russian Federation': 'Russia',
        'Bolivia (Plurinational State of)': 'Bolivia',
        'Democratic Republic of the Congo': 'Congo, Dem. Rep.',
        'Congo, Rep.': 'Congo',
        'Syrian Arab Republic': 'Syria',
        'Venezuela (Bolivarian Republic of)': 'Venezuela',
        'Slovak Republic': 'Slovakia',
        'Democratic People\'s Republic of Korea': 'North Korea',
        'Republic of Korea': 'South Korea',
        'Saint Kitts and Nevis': 'St. Kitts and Nevis',
        'Micronesia (Federated States of)': 'Micronesia, Fed. Sts.',
        'Micronesia': 'Micronesia, Fed. Sts',
        'Brunei Darussalam': 'Brunei',
        'Cote d\'Ivoire': 'Côte d\'Ivoire',
        'Saint Lucia': 'St. Lucia',
        'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines',
        'Republic of Moldova': 'Moldova',
        'Republic of North Macedonia': 'Macedonia, FYR',
        'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
        'Unted Kingdom': 'United Kingdom',
        'United Republic of Tanzania': 'Tanzania',
        'Swaziland': 'Eswatini',
        'Lao People\'s Democratic Republic': 'Lao',
        'Cabo Verde': 'Cape Verde',
        'Kyrgyz Republic': 'Kyrgyzstan'
    }, inplace=True)

# join data into one big table
df = pd.concat(df_tmp.values(), axis=1, keys=df_tmp.keys(), sort=False)

tmp = [x for x in range(2000, 2019) if len(df.xs(str(x), axis=1, level=1).columns) == len(df_tmp)]
print('We have complete set of data for years:')
print(tmp)

df_max = max(tmp)
df_latest = df.xs(str(df_max), axis=1, level=1)
print(f'Picking latest: {df_max}')

#pop = df.xs('2014', axis=1, level=1)['population']
#display(pop.loc[pop.isnull()])
#display(pop.filter(regex='Kyr', axis=0)).

In [ ]:
df_latest.describe()

In [ ]:
df_latest.loc['Poland']

In [ ]:
%matplotlib notebook

fig, ax = plt.subplots()

pop = df_latest['population']
pop = pop.loc[pop.notnull()]
pop = (pop / pop.max()) * 1024

sc = ax.scatter(df_latest['exp_health'], df_latest['income'], s=pop, alpha=0.5)

ax.set_xlabel(r'Health Expenditure (% of GDP)', fontsize=15)
ax.set_ylabel(r'GDP per capita', fontsize=15)
ax.set_title('Fun')

ax.grid(True)
fig.tight_layout()


annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->"))
annot.set_visible(False)

def update_annot(ind):

    pos = sc.get_offsets()[ind["ind"][0]]
    annot.xy = pos
    text = "{}, {}".format(" ".join(list(map(str,ind["ind"]))), 
                           " ".join([df_latest.index[n] for n in ind["ind"]]))
    annot.set_text(text)
    annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
    annot.get_bbox_patch().set_alpha(0.4)


def hover(event):
    vis = annot.get_visible()
    if event.inaxes == ax:
        cont, ind = sc.contains(event)
        if cont:
            update_annot(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                fig.canvas.draw_idle()

fig.canvas.mpl_connect("motion_notify_event", hover)

plt.show()

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np; np.random.seed(1)

x = df_latest['exp_health']
y = df_latest['income']
names = df_latest.index
c = df_latest['population']

norm = plt.Normalize(1,4)
cmap = plt.cm.RdYlGn

fig,ax = plt.subplots()
sc = plt.scatter(x,y,c=c, s=100, cmap=cmap, norm=norm)

annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->"))
annot.set_visible(False)

def update_annot(ind):

    pos = sc.get_offsets()[ind["ind"][0]]
    annot.xy = pos
    text = "{}, {}".format(" ".join(list(map(str,ind["ind"]))), 
                           " ".join([names[n] for n in ind["ind"]]))
    annot.set_text(text)
    annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
    annot.get_bbox_patch().set_alpha(0.4)


def hover(event):
    vis = annot.get_visible()
    if event.inaxes == ax:
        cont, ind = sc.contains(event)
        if cont:
            update_annot(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                fig.canvas.draw_idle()

fig.canvas.mpl_connect("motion_notify_event", hover)

plt.show()